# Getting Started with Segmentations

## In this notebook the following operations will be demonstrated:

1. Login and establish session to access the portal
2. List the datasets available on the portal
3. Upload a dataset
4. Access and download a dataset 
5. Delete a dataset


In [ ]:
import shapeworks as sw
from shutil import rmtree
from pathlib import Path

In [ ]:
from swcc.api import swcc_session
from swcc.models import Dataset
from swcc.models import (
    Dataset, GroomedSegmentation, OptimizedParticles,
    OptimizedShapeModel, Project, Segmentation, Subject
)

## Establish session and list the available datasets

In [ ]:
#api as a context manager
with swcc_session()  as session:
    token = session.login('iyerkrithika21@gmail.com', 'ilovegirder')
    session = swcc_session(token=token).__enter__()
    print([(d.id, d.name) for d in Dataset.list()])

In [ ]:
# For development, it is possible to generate a session outside of a context manager.
ctx = swcc_session(token=token)
session = ctx.__enter__()
print([(d.id, d.name) for d in Dataset.list()])

## Upload a datset
In order to upload the a dataset you need:
1. Segmentations/Images/Meshes 
2. Data file (xlxs)<br>
   a. One column each for segmentation/images/meshes
   b. The sheet has to be named -'data'
3. Project file (xlxs)-This is same the projct file used by ShapeWorksStudio<br>




In [ ]:
dataset = Dataset(
        name='ellipsoid_test',
        license='Users should acknowledge that the collection and processing of these images was supported by the National Institutes of Health',
        description='Unaligned ellipsoids that vary along the x-axis. \
                     These ellipsoids have randomized size, center, rotation, and x radii.\
                     Images and segmentation have spacing [1,1,2]',
        acknowledgement='NIH')

dataset.create()
data = dataset.load_data_spreadsheet(Path('../Output/ellipsoid/data.xlsx'))
project = dataset.add_project(Path('../Output/ellipsoid/project.xlsx'))


## We can also explore the data using the models

<h3>**How to add the anatomy type and how to address the other sheets from the project file?

In [ ]:
segmentation = next(data.segmentations)
print(f'Anatomy type: {segmentation.anatomy_type}') 

shape_models = next(project.shape_models)
shape_models.parameters

In [ ]:
#access individual parameters of shape model
print("Number of particles", shape_models.parameters["number_of_particles"])
print("Narrow band ", shape_models.parameters["narrow_band"])
print("Use normals ", shape_models.parameters["use_normals"])

## Given an ID, fetch data from the server

<h3>How to access shape files?**

In [ ]:
print([(d.id, d.name) for d in Dataset.list()])

In [ ]:
dataset_id = 56
Dataset.from_id(dataset_id)

## Delete a dataset

In [ ]:
# dataset = Dataset.from_name('ellipsoid8_3')
# dataset.delete()
# print([(d.id, d.name) for d in Dataset.list()])

## Download a dataset

In [ ]:
# Download a full dataset in bulk
dataset = Dataset.from_name('ellipsoid_test')
download_path = Path('../Output/downloads/ellipsoid/')
if download_path.exists():
    rmtree(str(download_path))
    
dataset.download(download_path)

<h3>Not clear how to use this ? **

In [ ]:
# We can also explore the data using the models
segmentation = next(subject.segmentations)
print(f'Anatomy type: {segmentation.anatomy_type}')

shape_models = next(project.shape_models)
shape_models.parameters



# The entities contain urls to download the associated files
print('Download url:')
print(segmentation.file.url)

print('')

# There is a helper method to download the file to a provided path
print(f"Downloaded {segmentation.file.download('downloads')}...")

print('')

# There is also a helper method on models to download all files attached to them
groomed = next(project.groomed_segmentations)
for download in groomed.download_files('downloads/groomed'):
    print(f'Downloaded {download}...')

